# Step 1: Auth

```markdown
Important Notes!!! 
- For this challenge you need to go to Azure AI Foundry portal and add a Bing Ressource as a connection. The name provided in the process is the one you need to reference in your .env file.
- Bing Groundedness only works with GPT-4o and not with GPT-4o-mini

More on Grounding with Bing: https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview

In [1]:
import os
from typing import Any, Callable, Set

# Azure AI Projects and authentication
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet, BingGroundingTool
from azure.identity import AzureCliCredential

# Azure Blob Storage client
from azure.storage.blob import BlobServiceClient

# Load environment variables from the .env file
from dotenv import load_dotenv
from pathlib import Path

# Construct the path to the .env file in the parent directory
env_path = Path().resolve().parent.parent / ".env"

# Load environment variables from the specified .env file
load_dotenv(dotenv_path=env_path)


# Retrieve keys from environment variables
BING_API_KEY = os.getenv("BING_API_KEY")
PROJECT_CONNECTION_STRING = os.getenv("Azure_AI_PROJECT_CONNECTION_STRING")
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
BING_CONNECTION_NAME_IN_AZURE_AI_FOUNDRY = os.getenv("BING_CONNECTION_NAME_IN_AZURE_AI_FOUNDRY")

if not BING_API_KEY:
    raise ValueError("BING_API_KEY is not set in the .env file.")
if not PROJECT_CONNECTION_STRING:
    raise ValueError("PROJECT_CONNECTION_STRING is not set in the .env file.")
if not AZURE_STORAGE_CONNECTION_STRING:
    raise ValueError("AZURE_STORAGE_CONNECTION_STRING is not set in the .env file.")
if not BING_CONNECTION_NAME_IN_AZURE_AI_FOUNDRY:
    raise ValueError("AZURE_STORAGE_CONNECTION_STRING is not set in the .env file.")

print("Environment variables loaded successfully.")

Environment variables loaded successfully.


# Step 2: Create a Project Client

In [2]:
client = AIProjectClient.from_connection_string(
        credential=AzureCliCredential(),
        conn_str=PROJECT_CONNECTION_STRING,
        
    )

# Step 3: Create an Agent with the Grounding with Bing search tool enabled

```markdown
Please note that BING_CONNECTION_NAME_IN_AZURE_AI_FOUNDRY needs to be added as a connection in Azure AI Foundry first

In [3]:
bing_connection = client.connections.get(connection_name=BING_CONNECTION_NAME_IN_AZURE_AI_FOUNDRY)
conn_id = bing_connection.id

print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

/subscriptions/4c9216b8-3c30-4c2f-8ced-0837fea45954/resourceGroups/basic-agent-setup-713/providers/Microsoft.MachineLearningServices/workspaces/ffollonier-rag-project-713/connections/basicbing


In [ ]:

# ---------------------------------------------------------------------------
# Prepare the tool for the Azure AI Agent service.
# We wrap our search function in a FunctionTool and add it to a ToolSet.
# ---------------------------------------------------------------------------


"""
Runs the Azure AI Agent pipeline.

The agent uses the Bing Search API tool to fetch the latest AI news.
It creates a simple conversation thread where the user message triggers the tool.
"""
print("Starting the Azure AI Agent pipeline...")
# Create the Azure AI Projects client with a connection string and default credential.


#<-- This is a simple agent that fetches the latest AI news. -->
#<-- It uses the Bing Search API tool to get the news. -->
#<-- Uncomment the rows to allow the agent store the news in Azure Blob Storage. -->

# Instructions for the agent
instructions = """ 
You are a helpful agent that takes user questions and searches the web using Bing Search"""

with client:
    # Create the agent using a chosen model (e.g., gpt-4o)
    agent = client.agents.create_agent(
        model="gpt-4o",
        name="simple-news-agent",
        instructions=instructions,
        tools=bing.definitions
    )
    print(f"Created agent with ID: {agent.id}")

    # Start a new conversation thread
    thread = client.agents.create_thread()
    print(f"Created thread with ID: {thread.id}")

    # Create an initial user message
    message = client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Show me the latest AI news."
    )
    print(f"User message created with ID: {message.id}")

    # Process the conversation
    run = client.agents.create_and_process_run(
        thread_id=thread.id,
        assistant_id=agent.id
    )
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    else:
        # Retrieve and log all messages from the conversation
        messages = client.agents.list_messages(thread_id=thread.id)
        for msg in messages["data"]:
            if msg["role"] == "assistant":
                print("Assistant response:")
                for part in msg["content"]:
                    if part["type"] == "text":
                        print(part["text"]["value"])

    # Cleanup the agent after the run
    client.agents.delete_agent(agent.id)
    print("Agent deleted successfully.")



Starting the Azure AI Agent pipeline...
Created agent with ID: asst_Sk5V7ZrjJPjzT6qXr6uZzJJH
Created thread with ID: thread_ab7sLNEhNbjAnL1bQZsFhGUr
User message created with ID: msg_mBBsnZn0RbeAhqMR7WXb5hEp
Run finished with status: RunStatus.COMPLETED
Assistant response:
It seems there was an issue storing the information in Azure Blob Storage. Please ensure that the storage configuration is correctly set up and try again later. 

However, I've provided a summary of the latest AI news, including OpenAI's new model release, Tencent's AI advancements, and the expansion of OpenAI's Sora service in Europe. Let me know if there's anything else you need!
Assistant response:
Here are the latest developments in AI:

1. **OpenAI Launches GPT-4.5 'Orion'**: OpenAI has unveiled its largest AI model to date, GPT-4.5. This model continues the trend of increasing computational power and data usage for improved performance. It's available to users of ChatGPT Pro and OpenAI's API. Although it perfor